In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from pandas_profiling import ProfileReport

import statsmodels.api as sm

from statsmodels.formula.api import ols

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.special import inv_boxcox
import scipy.stats as stats

In [2]:
df = pd.read_csv('data/data_after_clean.csv')

In [20]:
df

,Unnamed: 0,date,price,qty,sell_id,sell_cat,item,year,holiday,is_weekend,is_schoolbreak,avg_temperature,is_outdoor,is_holiday
0,0,2012-01-01,15.50,46,1070,0,BURGER,2012,5,1,0,24.8,0,1
1,1,2012-01-01,12.73,22,2051,2,BURGER,2012,5,1,0,24.8,0,1
2,2,2012-01-01,12.73,22,2051,2,COKE,2012,5,1,0,24.8,0,1
3,3,2012-01-01,12.75,18,2052,2,BURGER,2012,5,1,0,24.8,0,1
4,4,2012-01-01,12.75,18,2052,2,LEMONADE,2012,5,1,0,24.8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10779,10835,2015-09-10,11.50,18,2052,2,BURGER,2015,6,0,0,73.4,1,0
10780,10836,2015-09-10,11.50,18,2052,2,LEMONADE,2015,6,0,0,73.4,1,0
10781,10837,2015-09-10,11.26,50,2053,2,BURGER,2015,6,0,0,73.4,1,0
10782,10838,2015-09-10,11.26,50,2053,2,COKE,2015,6,0,0,73.4,1,0


In [4]:
df_1070 = df[df['sell_id']==1070]
df_2051 = df[df['sell_id']==2051]
df_2052 = df[df['sell_id']==2052]
df_2053 = df[df['sell_id']==2053]

In [12]:
df_1070_unique = df_1070.drop_duplicates(subset=['date'])
df_2053_unique = df_2053.drop_duplicates(subset=['date'])
df_2052_unique = df_2052.drop_duplicates(subset=['date'])
df_2051_unique = df_2051.drop_duplicates(subset=['date'])

In [28]:
feat_df = df_1070_unique[['holiday','is_weekend','is_schoolbreak','avg_temperature','is_outdoor','is_holiday']]

In [29]:
feat_df

,holiday,is_weekend,is_schoolbreak,avg_temperature,is_outdoor,is_holiday
0,5,1,0,24.8,0,1
8,5,0,0,24.8,0,1
16,5,0,0,32.0,1,1
24,6,0,0,32.0,1,0
32,6,0,0,24.8,0,0
...,...,...,...,...,...,...
10744,6,1,0,75.2,1,0
10752,6,0,0,77.0,1,0
10760,6,0,0,73.4,1,0
10768,6,0,0,71.6,1,0


In [36]:
feat_df.to_csv('data/feature.csv')

In [30]:
df_1070_unique

,Unnamed: 0,date,price,qty,sell_id,sell_cat,item,year,holiday,is_weekend,is_schoolbreak,avg_temperature,is_outdoor,is_holiday
0,0,2012-01-01,15.5,46,1070,0,BURGER,2012,5,1,0,24.8,0,1
8,8,2012-01-02,15.5,70,1070,0,BURGER,2012,5,0,0,24.8,0,1
16,16,2012-01-03,15.5,62,1070,0,BURGER,2012,5,0,0,32.0,1,1
24,24,2012-01-04,15.5,88,1070,0,BURGER,2012,6,0,0,32.0,1,0
32,32,2012-01-05,15.5,104,1070,0,BURGER,2012,6,0,0,24.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10744,10800,2015-09-06,14.5,94,1070,0,BURGER,2015,6,1,0,75.2,1,0
10752,10808,2015-09-07,14.5,98,1070,0,BURGER,2015,6,0,0,77.0,1,0
10760,10816,2015-09-08,14.5,90,1070,0,BURGER,2015,6,0,0,73.4,1,0
10768,10824,2015-09-09,14.5,96,1070,0,BURGER,2015,6,0,0,71.6,1,0


In [24]:
df_2051_unique_without_covariates = df_2051_unique[['date','price','qty','sell_id']]
df_2052_unique_without_covariates = df_2052_unique[['date','price','qty','sell_id']]
df_2053_unique_without_covariates = df_2053_unique[['date','price','qty','sell_id']]

In [27]:
df_2051_unique_without_covariates

,date,price,qty,sell_id
1,2012-01-01,12.73,22,2051
9,2012-01-02,12.73,22,2051
17,2012-01-03,12.73,26,2051
25,2012-01-04,12.73,26,2051
33,2012-01-05,12.73,36,2051
...,...,...,...,...
10745,2015-09-06,12.19,32,2051
10753,2015-09-07,12.19,36,2051
10761,2015-09-08,12.19,30,2051
10769,2015-09-09,12.19,34,2051


In [17]:
df_2052_unique

,Unnamed: 0,date,price,qty,sell_id,sell_cat,item,year,holiday,is_weekend,is_schoolbreak,avg_temperature,is_outdoor,is_holiday
3,3,2012-01-01,12.75,18,2052,2,BURGER,2012,5,1,0,24.8,0,1
11,11,2012-01-02,12.75,16,2052,2,BURGER,2012,5,0,0,24.8,0,1
19,19,2012-01-03,12.75,12,2052,2,BURGER,2012,5,0,0,32.0,1,1
27,27,2012-01-04,12.75,24,2052,2,BURGER,2012,6,0,0,32.0,1,0
35,35,2012-01-05,12.75,24,2052,2,BURGER,2012,6,0,0,24.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10747,10803,2015-09-06,11.50,26,2052,2,BURGER,2015,6,1,0,75.2,1,0
10755,10811,2015-09-07,11.50,20,2052,2,BURGER,2015,6,0,0,77.0,1,0
10763,10819,2015-09-08,11.50,26,2052,2,BURGER,2015,6,0,0,73.4,1,0
10771,10827,2015-09-09,11.50,18,2052,2,BURGER,2015,6,0,0,71.6,1,0


In [19]:
df_2051_unique

,Unnamed: 0,date,price,qty,sell_id,sell_cat,item,year,holiday,is_weekend,is_schoolbreak,avg_temperature,is_outdoor,is_holiday
1,1,2012-01-01,12.73,22,2051,2,BURGER,2012,5,1,0,24.8,0,1
9,9,2012-01-02,12.73,22,2051,2,BURGER,2012,5,0,0,24.8,0,1
17,17,2012-01-03,12.73,26,2051,2,BURGER,2012,5,0,0,32.0,1,1
25,25,2012-01-04,12.73,26,2051,2,BURGER,2012,6,0,0,32.0,1,0
33,33,2012-01-05,12.73,36,2051,2,BURGER,2012,6,0,0,24.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10745,10801,2015-09-06,12.19,32,2051,2,BURGER,2015,6,1,0,75.2,1,0
10753,10809,2015-09-07,12.19,36,2051,2,BURGER,2015,6,0,0,77.0,1,0
10761,10817,2015-09-08,12.19,30,2051,2,BURGER,2015,6,0,0,73.4,1,0
10769,10825,2015-09-09,12.19,34,2051,2,BURGER,2015,6,0,0,71.6,1,0


In [31]:
demand_df = pd.DataFrame()
demand_df[1070] = df_1070_unique['qty'].values
demand_df[2051] = df_2051_unique['qty'].values
demand_df[2052] = df_2052_unique['qty'].values
demand_df[2053] = df_2053_unique['qty'].values

In [32]:
demand_df

,1070,2051,2052,2053
0,46,22,18,30
1,70,22,16,34
2,62,26,12,28
3,88,26,24,42
4,104,36,24,58
...,...,...,...,...
1343,94,32,26,54
1344,98,36,20,50
1345,90,30,26,50
1346,96,34,18,54


In [33]:
demand_df.to_csv('data/demand.csv')

In [34]:
price_df = pd.DataFrame()
price_df[1070] = df_1070['price'].values
price_df[2051] = df_2051_unique['price'].values
price_df[2052] = df_2052_unique['price'].values
price_df[2053] = df_2053_unique['price'].values


In [23]:
price_df

,1070,2051,2052,2053
0,15.5,12.73,12.75,12.60
1,15.5,12.73,12.75,12.60
2,15.5,12.73,12.75,12.60
3,15.5,12.73,12.75,12.60
4,15.5,12.73,12.75,12.60
...,...,...,...,...
1343,14.5,12.19,11.50,11.26
1344,14.5,12.19,11.50,11.26
1345,14.5,12.19,11.50,11.26
1346,14.5,12.19,11.50,11.26


In [35]:
price_df.to_csv('data/price.csv')